In [1]:
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker

%run basics_sensorSignal.py
%matplotlib
matplotlib.use("Qt5Agg")

Using matplotlib backend: MacOSX


global variables

In [2]:
# sensor characteristics fixed
pKa = 9.25                 # pKa of ammonia
phmin, phmax = 0, 15       # dynamic range of pH sensor
step_ph = 0.01
ph_deci = 2                # decimals for sensor sensitivity
ph_res = 1e-5              # resolution of the pH sensor 
t_plateau = 10             # int, time for one pH value in seconds

sig_max = 400              # maximal signal response at maximal NH4+ concentration in mV 
sig_bgd = 5.0              # background signal / offset in mV at 0M NH4+
tresp_pH = .1              # response time of pH sensor in seconds
E0 = (sig_bgd)/1000        # zero potential of the reference electrode
Temp = 25                  # measurement temperature in degC
tsteps = 1e-3              # time steps for pH and NH3 sensor (theory)

# electrochemical NH3 sensor
c_nh4_ppm = 230.             # concentration of ammonium ion at the pKa; in ppm
anh3_min, anh3_max = 0, 100  # dynamic range of the NH3 sensor in %
anh3_step = .01              # steps for concentration range
sigNH3_max = 0.09            # maximal signal at maximal NH3 concentration in mV (pH=1) 
sigNH3_bgd = 0.02            # background signal / offset in mV at 0M NH3
tresp_nh3 = .5               # response time of the NH3 sensor in seconds
nh3_res = 1e-9               # resolution of the NH3 sensor 
smpg_rate = 1.               # sampling rate of NH3 sensor in seconds

# --------------------------------------------------------------------------------------
# USER INPUT
# --------------------------------------------------------------------------------------
# time intervals pH fluctuations - step function
pH_steps = list(np.arange(phmin, phmax))  # steps of pH changes; linearly or randomly

random = input('Select: (1) Random pH gradient or (2) ramp function? >')
if random == '1' or random == 'random':
    np.random.shuffle(pH_steps)
elif random == '2' or random == 'ramp':
    pH_steps = pH_steps
else:
    raise ValueError('Choose between 1 and 2')
print('Targeted pH changes:', pH_steps)

Select: (1) Random pH gradient or (2) ramp function? >2
Targeted pH changes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [3]:
# collecting parameter within a dictionary
sensor_ph = dict({'E0': E0, 't90': tresp_pH, 'resolution': ph_res, 'time steps': tsteps, 
                  'background signal': sig_bgd, 'sensitivity': ph_deci})
sensor_nh3 = dict({'pH range': (phmin, phmax, step_ph), 'sensitivity': ph_deci, 'pKa': pKa,
                   'response time': tresp_nh3, 'resolution': nh3_res,
                   'time steps': tsteps, 'nh3 range': (anh3_min, anh3_max, anh3_step), 
                   'signal min': sigNH3_bgd, 'signal max': sigNH3_max})

para_meas = dict({'Temperature': Temp, 'Plateau time': t_plateau, 'pH steps': pH_steps,
                 'sampling rate': smpg_rate, 'GGW concentration': c_nh4_ppm})

# main simulation / modeling function 
[dtarget, dsens_calib, dsens_rec, 
 dfig, df_tan] = tan_simulation(sensor_ph=sensor_ph, para_meas=para_meas, 
                                sensor_nh3=sensor_nh3, plot='results')